In [3]:
from shapely.geometry import Point, Polygon
import pandas as pd
import osmnx as ox

def extract_osm_data(place_name, tags, output_file, level_map=None):
    
    # Download the OpenStreetMap data for the area
    pois = ox.geometries_from_place(place_name, tags=tags)

    # Convert the data to a pandas DataFrame
    df = pd.DataFrame(pois, columns=['name', 'addr:street', 'addr:city', 'addr:postcode', 'employees','amenity', 'geometry'])

    # Map each point of interest to a level based on the 'amenity' tag, if a level_map is provided
    if level_map:
        df['level'] = df['amenity'].map(level_map)

    # Filter out any rows where the 'level' column is NaN
        df = df.dropna(subset=['level'])

    # Apply the 'extract_coordinates' function to create separate 'x' and 'y' columns
    def extract_coordinates(row):
        geom = row['geometry']
        if isinstance(geom, Point):
            x, y = geom.coords[0]
        elif isinstance(geom, Polygon):
            centroid = geom.centroid
            x, y = centroid.x, centroid.y
        else:
            raise ValueError(f"Unsupported geometry type: {type(geom)}")
        return (x, y)

    df['x'], df['y'] = zip(*df.apply(extract_coordinates, axis=1))

    # Save the resulting DataFrame to a CSV file
    df.to_csv(output_file, index=False)

    return df

In [5]:
df_business =extract_osm_data("Casablanca, Morocco", {'building': ['commercial', 'industrial', 'office', 'retail'], 'amenity': ['coworking_space', 'conference_centre', 'courthouse', 'office', 'public_building']}, 'businesses.csv', level_map=None)


In [4]:
df_schools = extract_osm_data("Casablanca, Morocco", {'amenity': ['school', 'university']} , 'education.csv', level_map={'school': 'school', 'university': 'university'})